In [ ]:
from pathlib import Path
import torch
from effdet import create_model, create_loader, create_evaluator
from effdet.data import resolve_input_config
try:
    from timm.layers import set_layer_config
except ImportError:
    from timm.models.layers import set_layer_config
from ml_carbucks.utils.coco import create_dataset_custom
from effdet.anchors import Anchors, AnchorLabeler

BATCH_SIZE = 16
IMG_SIZE = 320
NUM_CLASSES = None
EPOCHS = 400
extra_args = dict(image_size=(IMG_SIZE, IMG_SIZE))



In [ ]:
# TRAINING

bench_train = create_model(
    model_name='tf_efficientdet_d2', 
    bench_task='train',
    num_classes=NUM_CLASSES,
    pretrained=True,
    redundant_bias=None,
    soft_nms=None,
    checkpoint_path='',
    checkpoint_ema=False,
    **extra_args,
)
model_train_config = bench_train.config
labeler = AnchorLabeler(
    Anchors.from_config(model_train_config),
    model_train_config.num_classes,
    match_threshold=0.5,
)

train_dataset = create_dataset_custom(
    name="val",
    # img_dir=Path("/home/bachelor/ml-carbucks/data/mscoco/val2017"),
    # ann_file=Path("/home/bachelor/ml-carbucks/data/mscoco/annotations/instances_val2017.json"),
    img_dir=Path("/home/bachelor/ml-carbucks/data/car_dd/images/val"),
    ann_file=Path("/home/bachelor/ml-carbucks/data/car_dd/instances_val.json"),
)

train_input_config = resolve_input_config({}, model_train_config)
train_loader = create_loader(
    train_dataset,
    input_size=train_input_config['input_size'],
    batch_size=BATCH_SIZE,
    use_prefetcher=True,
    interpolation=train_input_config['interpolation'],
    mean=train_input_config['mean'],
    std=train_input_config['std'],
    num_workers=4,
    pin_mem=False,
    anchor_labeler=labeler,
)

for param in bench_train.model.backbone.parameters():
    param.requires_grad = False

optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, bench_train.parameters()), lr=1e-4
)

bench_train = bench_train.cuda()
# set_layer_config(bench_train, 'fuse_bn', True)
bench_train.train()
for epoch in range(EPOCHS):
    for batch_idx, (input, target) in enumerate(train_loader):
        output = bench_train(input, target)
        loss = output['loss']
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item():.4f}")


train_state_dict = bench_train.model.state_dict()


In [ ]:
# import cloudpickle as cpkl
# cpkl.dump(train_state_dict, open("effdet_14_10_2025.pkl", "wb"))
torch.save(train_state_dict, "effdet_14_10_2025.pth")

In [ ]:
# EVALUATION

bench_pred = create_model(
    model_name='tf_efficientdet_d2', 
    bench_task='predict',
    num_classes=NUM_CLASSES,
    pretrained=True,
    redundant_bias=None,
    soft_nms=None,
    checkpoint_path='',
    checkpoint_ema=False,
    **extra_args,
)
# bench_pred.model.load_state_dict(train_state_dict)
model_pred_config = bench_pred.config
bench_pred = bench_pred.cuda()

val_dataset = create_dataset_custom(
    name="val",
    img_dir=Path("/home/bachelor/ml-carbucks/data/mscoco/val2017"),
    ann_file=Path("/home/bachelor/ml-carbucks/data/mscoco/annotations/instances_val2017.json"),
    # img_dir=Path("/home/bachelor/ml-carbucks/data/car_dd/images/val"),
    # ann_file=Path("/home/bachelor/ml-carbucks/data/car_dd/instances_val.json"),
)

input_config = resolve_input_config({}, model_pred_config)
loader = create_loader(
    val_dataset,
    input_size=input_config['input_size'],
    batch_size=BATCH_SIZE,
    use_prefetcher=True,
    interpolation=input_config['interpolation'],
    mean=input_config['mean'],
    std=input_config['std'],
    num_workers=4,
    pin_mem=False
)
evaluator = create_evaluator("coco", val_dataset, pred_yxyx=False)
bench_pred.eval()


with torch.no_grad():
    for i, (input, target) in enumerate(loader):
        output = bench_pred(input, img_info=target)
        evaluator.add_predictions(output, target)

        if i % 10 == 0:
            print(f"Eval {i}/{len(loader)}")

metrics = evaluator.evaluate()
print(metrics)

# THIS IS THE Xth ATTEMPT TO TRAIN/DEBUG IT

In [2]:
from effdet import EfficientDet, create_loader
from effdet.bench import DetBenchTrain
from effdet.config import get_efficientdet_config
from effdet.anchors import Anchors, AnchorLabeler
from effdet.data import resolve_input_config

from ml_carbucks import DATA_CAR_DD_DIR
from ml_carbucks.utils.coco import create_dataset_custom
from ml_carbucks.utils.logger import setup_logger
from ml_carbucks.utils.inference import plot_img_pred as ppp


IMG_SIZE = 320
BATCH_SIZE = 16
NUM_CLASSES = 3  # this time no background class is included into counting

logger = setup_logger("efficient_det_v2")

config = get_efficientdet_config(model_name="tf_efficientdet_d2")
setattr(config, "image_size", (IMG_SIZE, IMG_SIZE))

model = EfficientDet(config, pretrained_backbone=True)

model.reset_head(num_classes=NUM_CLASSES)

create_auto_labeler = True

bench_train = DetBenchTrain(model, create_labeler=create_auto_labeler)
model_train_config = bench_train.config

if create_auto_labeler:
    labeler = None
else:
    labeler = AnchorLabeler(
        Anchors.from_config(model_train_config),
        model_train_config.num_classes,
        match_threshold=0.25,
    )

train_dataset = create_dataset_custom(
    name="train",
    img_dir=DATA_CAR_DD_DIR / "images" / "train",
    ann_file=DATA_CAR_DD_DIR / "instances_train_curated.json",
)

train_input_config = resolve_input_config({}, model_train_config)

train_loader = create_loader(
    train_dataset,
    input_size=train_input_config["input_size"],
    batch_size=BATCH_SIZE,
    use_prefetcher=True,
    interpolation=train_input_config["interpolation"],
    mean=train_input_config["mean"],
    std=train_input_config["std"],
    num_workers=4,
    pin_mem=False,
    anchor_labeler=labeler,
)


In [ ]:
batch = next(iter(train_loader))

imgs, targets = batch
print(f"Images shape: {imgs.shape}")
ii = 3
ppp(imgs[ii], targets["bbox"][ii], coords="xyxy")

In [3]:
import torch
from tqdm import tqdm
from ml_carbucks.utils.coco import CocoStatsEvaluator
EPOCHS = 10
LR = 5e-4

train_evaluator = CocoStatsEvaluator(train_dataset, distributed=False, pred_yxyx=False)
optimizer = torch.optim.AdamW(bench_train.parameters(), lr=LR, weight_decay=1e-4)
bench_train = bench_train.cuda()

for epoch in range(EPOCHS):

    bench_train.train()
    for input, target in tqdm(train_loader):
        output = bench_train(input, target)
        loss = output["loss"]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    bench_train.eval()
    with torch.no_grad():
        for input, target in tqdm(train_loader):
            output = bench_train(input, target)
            train_evaluator.add_predictions(output['detections'], target)

    stats = train_evaluator.evaluate()
    stats = [round(s, 4) for s in stats]
    train_evaluator.reset()

    logger.info(f"map: {stats[0]}, map50: {stats[1]}")
    
        

100%|██████████| 176/176 [00:12<00:00, 14.61it/s]


INFO efficient_det_v2 19:01:26 | map: 0.0008, map50: 0.0042


100%|██████████| 176/176 [00:12<00:00, 14.57it/s]


INFO efficient_det_v2 19:02:19 | map: 0.0005, map50: 0.0028


100%|██████████| 176/176 [00:12<00:00, 14.49it/s]


INFO efficient_det_v2 19:03:12 | map: 0.0012, map50: 0.006


 36%|███▋      | 64/176 [00:12<00:21,  5.19it/s]


KeyboardInterrupt: 